In [ ]:
from gensim import corpora
from gensim.models import LdaModel
import json
from data_analysis.OpeningLDA import OpeningLDA
from classes.QueryHandler import QueryHandler

## LDA

In [ ]:
#opening_lda = OpeningLDA(num_passes=5, print_debug=True) 

#print(len(opening_lda.texts.keys()))
#opening_lda.lda_model.print_topics(num_words=15)

#opening_lda.save('lda')


In [ ]:
opening_lda = OpeningLDA.load("lda")

In [ ]:
opening_lda.__construct_topic_doc_matrix__()

In [ ]:
query_handler = QueryHandler()
query_handler.opening_lda = opening_lda

sample_query1 = { 
                 "message": "capture the center, develop light square bishop",
                 "color": "white" 
}
query_handler.handle_user_query(sample_query1, debug=True)

## GPT

In [ ]:
from data_analysis.OpeningGPT import OpeningGPT
from scraper.Paths import Paths

opening_gpt = OpeningGPT()


### Precomputing summaries for all ECOs 

In [ ]:
opening_gpt.precompute_all_summaries(print_debug=True, temperature=0.8, overwrite_existing=True)

In [ ]:
opening_gpt.randomize_all_summaries(print_debug=True)